### Descarga de capas de mapa de cobertura 4G del Instituto Federal de Telecomunicaciones

In [1]:
import requests
import geopandas as gpd
import pandas as pd
from bs4 import BeautifulSoup
import os

In [2]:
#Crear directorio para guardar los archivos
os.makedirs("geopaquetes", exist_ok=True)
#Cambiar al directorio
os.chdir("geopaquetes")

Paso 1: Se obtiene el código html de la página en donde se encuentra el mapa.

In [3]:
url="https://felt.com/map/Mapa-interactivo-de-cobertura-4G-IFT-nkwEcoI4S9BSa7vgFt6uctD?loc=20.0624,-98.7601,11.6z"

In [4]:
#Obtener el mapa
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
# obtener div id="felt-data"
div = soup.find("div", {"id": "felt-data"})
#Eliminar textos
div = str(div).replace('<div id="felt-data" style="display: none">', '')
div = div.replace('\n  </div>', '')
div=div.replace('\n','')

In [6]:
#Transformar a json
data = json.loads(div)
data
#Salvar a un archivo
with open('data.json', 'w') as f:
    json.dump(data, f)


NameError: name 'json' is not defined

In [ ]:
data.keys()

In [ ]:
data["layerGroups"][0].keys()

Paso 2: Se obtienen los nombres y urls de la información de cada capa del mapa

In [ ]:
#Obtener nombres y urls
nombres=[]
urls=[]
for i in range(len(data["layerGroups"])):
    nombres.append(data["layerGroups"][i]["layers"][0]["normalized"]["layername"])
    urls.append(data["layerGroups"][i]["layers"][0]["index_json_url"])

In [ ]:
#Crear tabla
tabla = pd.DataFrame({"nombre":nombres, "url":urls})
#Eliminar último elemento
tabla = tabla[:-1]
#Renombrar último registro como "Movistar"
tabla["nombre"][3]="Movistar"
tabla["nombre"][0]="ATT"
tabla

Paso 3: Se obtienen las urls de los datos de cada capa y se descargan los archivos

In [ ]:
#Obtener las urls de la data de los mapas
urls_data=[]
gdf=gpd.GeoDataFrame()
for i in range(len(tabla)):
    url = tabla["url"][i]
    response = requests.get(url)
    data= response.json()
    urls_data.append(data["datasets"][0]["data_url"])
    #descargar el geopackage de cada url con el nombre de la capa
    nombre_archivo = tabla["nombre"][i]
    write_path = f"{nombre_archivo}.gpkg"
    with open(write_path, 'wb') as f:
        f.write(requests.get(data["datasets"][0]["data_url"]).content)

Paso 4: Se cargan los archivos geopackage y se concatenan en un solo archivo

In [ ]:
gdfs = []
#Cargar datos
for i in range(len(tabla)):
    nombre_archivo = tabla['nombre'][i]
    gdf = gpd.read_file(f"{nombre_archivo}.gpkg")
    #Incluir el nombre de la capa
    gdf["nombre"] = tabla["nombre"][i]
    gdfs.append(gdf)
    
# Concatenar los geodataframes
gdf_final = pd.concat(gdfs, ignore_index=True)

In [ ]:
gdf_final

In [ ]:
type(gdf_final)

Paso 5: Se guarda el archivo final en formato geopackage

In [ ]:
#Salvar como geopackage
gdf_final.to_file("cobertura_4g.gpkg", driver="GPKG",crs="EPSG:4326")